# Set up Model Predictive Control Using Do-MPC Package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import casadi as cas
import do_mpc

from cas_models.continuous_time.models import StateSpaceModelCT
from cas_models.transformations import connect_systems
from cas_models.discrete_time.models import StateSpaceModelDTFromCT
from cas_models.discrete_time.simulate import make_n_step_simulation_function_from_model

from feed_conc_ctrl.models import MixingTankModelCT, RatioControlledFlowMixerCT

## Example: Two-Tank Feed Blending System

```none
        ┌────────┐        
  ─────►┤ Tank 1 │        
        │        │        
        │        ├──┐     
        └────────┘  │     
                    │     
                    │     
                    ├────►
        ┌────────┐  │     
  ─────►┤ Tank 2 │  │     
        │        │  │     
        │        ├──┘     
        └────────┘        
```

States
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass

Inputs:
 1. Tank 1 inflow rate
 2. Tank 1 inflow concentration
 3. Tank 1 outflow rate
 4. Tank 2 inflow rate
 5. Tank 2 inflow concentration
 6. Tank 2 outflow rate

Outputs:
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 1 outflow concentration
 4. Tank 2 level
 5. Tank 2 mass
 6. Tank 2 outflow concentration
 7. Mixer outflow rate
 8. Mixer outflow concentration

In [2]:
def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

## Construct Mixing Tank System Model using Casadi-Models Package

In [3]:
# Test new ratio controller
mixer = RatioControlledFlowMixerCT(2)
assert (
    str(mixer) == (
        "RatioControlledFlowMixerCT("
        "f=Function(f:(t,x[0],u[4])->(rhs[0]) SXFunction), "
        "h=Function(h:(t,x[0],u[4])->(y[3]) SXFunction), "
        "n=0, nu=4, ny=3, params={}, name='FlowMixerModel', "
        "input_names=['r_1', 'conc_in_1', 'v_dot_out', 'conc_in_2'], "
        "state_names=[], "
        "output_names=['v_dot_in_1', 'v_dot_in_2', 'conc_out'])")
)

t = 0.0
x = []

# Equal inflow concentrations, 50:50 ratio
u = [0.5, 10.0, 2.0, 10.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [1, 1, 10])

# Equal inflow concentrations, 20:80 ratio
u = [0.2, 10.0, 2.0, 10.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [0.4, 1.6, 10])

# Different concentrations, 50:50 ratio
u = [0.5, 10.0, 2.0, 0.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [1, 1, 5])

#  Mixer wit 3 inputs
mixer = RatioControlledFlowMixerCT(3)
assert (
    str(mixer) == (
        "RatioControlledFlowMixerCT("
        "f=Function(f:(t,x[0],u[6])->(rhs[0]) SXFunction), "
        "h=Function(h:(t,x[0],u[6])->(y[4]) SXFunction), "
        "n=0, nu=6, ny=4, params={}, name='FlowMixerModel', "
        "input_names=['r_1', 'conc_in_1', 'r_2', 'conc_in_2', 'v_dot_out', 'conc_in_3'], "
        "state_names=[], "
        "output_names=['v_dot_in_1', 'v_dot_in_2', 'v_dot_in_3', 'conc_out'])")
)

u = [0.3, 10.0, 0.4, 20.0, 2.0, 30.0]
y = mixer.h(t, x, u)
assert np.allclose(
    np.array(y).flatten(), 
    [0.3 * 2, 0.4 * 2, 0.3 * 2, (0.3 * 10 + 0.4 * 20 + 0.3 * 30)]
)

In [ ]:
D = 5  # tank diameter [m]
n_tanks = 2
tank_names = [f"tank_{i+1}" for i in range(n_tanks)]

# Initialize tank system models
systems = [MixingTankModelCT(D=D, name=name) for name in tank_names]

# Add a flow mixer to join flows from two tanks
systems.append(RatioControlledFlowMixerCT(2, name="mixer"))

# Check inputs and outputs
for sys in systems:
    print_sys_dimensions(sys)

In [ ]:
# Connect all systems together
connections = {
    'mixer_conc_in_1': 'tank_1_conc_out',
    'mixer_conc_in_2': 'tank_2_conc_out',
    'tank_1_v_dot_out': 'mixer_v_dot_in_1',
    'tank_2_v_dot_out': 'mixer_v_dot_in_2',
}

model_class = StateSpaceModelCT
feed_tanks_system = connect_systems(
    systems, connections, model_class, name="tank_system_21", verbose_names=True
) 
print_sys_dimensions(feed_tanks_system)

## Control Model Design

**States**
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass

**Manipulated Variables (MVs)**
 1. Tank 1 inflow rate
 2. Tank 2 inflow rate
 3. Mixer inflow ratio

where mixer inflow ratio = tank 1 outflow rate / 
    (tank 1 outflow rate + tank 2 outflow rate)

**Unmeasured Disturbances**
 1. Tank 1 inflow concentration
 2. Tank 2 inflow concentration
 3. Mixer outflow rate

**Controlled Variables (CVs)**
 1. Tank 1 level
 2. Tank 1 outflow concentration
 3. Tank 2 level
 4. Tank 2 outflow concentration
 5. Mixer outflow concentration



## Construct Do-MPC Model from Casadi System Model

In [6]:
system = feed_tanks_system
control_design = {
    "manipulated_variables": ['tank_1_v_dot_in', 'tank_2_v_dot_in', 'mixer_r_1'],
    "unmeasured_disturbances": ['tank_1_conc_in', 'tank_2_conc_in', 'mixer_v_dot_out'],
    "controlled_variables": [
        'tank_1_L', 'tank_1_conc_out', 'tank_2_L', 'tank_2_conc_out', 'mixer_conc_out'
    ],
}

# Check all inputs accounted for
assert set(system.input_names) == set(
    control_design["manipulated_variables"] + control_design["unmeasured_disturbances"]
)

model_type = 'continuous'
model = do_mpc.model.Model(model_type)

# Add manipulated variables (MVs)
for name in control_design["manipulated_variables"]:
    model.set_variable(var_type='_u', var_name=name, shape=(1, 1))

# Add all state variables in same order as system
for name in system.state_names:
    model.set_variable(var_type='_x', var_name=name, shape=(1, 1))

# Augment model with additional states for unmeasured disturbances
for name in control_design["unmeasured_disturbances"]:
    model.set_variable(var_type='_x', var_name=name, shape=(1, 1))

# TODO: How to add measured disturbances?



In [7]:
# METHOD 1 - using regular states and inputs attributes

t = 0  # assume time invariant
states = cas.vcat(model.x[system.state_names])

inputs = []
for name in system.input_names:
    if name in control_design['manipulated_variables']:
        inputs.append(model.u[name])
    elif name in control_design.get('measured_disturbances', []):
        inputs.append(model.x[name])
    elif name in control_design.get('unmeasured_disturbances', []):
        inputs.append(model.x[name])
inputs = cas.vcat(inputs)
# TODO: Deal with params

In [8]:
# # Alternative method

# # Build state vector using INTERNAL symbols
# states = cas.vertcat(*[model._x[name] for name in system.state_names])

# # Build input vector using INTERNAL symbols
# inputs = []
# for name in system.input_names:
#     if name in control_design['manipulated_variables']:
#         inputs.append(model._u[name])
#     elif name in control_design.get('unmeasured_disturbances', []):
#         inputs.append(model._x[name])
# inputs = cas.vertcat(*inputs)

In [ ]:
# Generate expressions from CasADi model functions
rhs = system.f(t, states, inputs)
outputs = system.h(t, states, inputs)

# Set righthand-side expressions for system states
for i, name in enumerate(system.state_names):
    model.set_rhs(name, rhs[i])

# Set righthand-side expressions for unmeasured disturbances
for i, name in enumerate(control_design['unmeasured_disturbances']):
    model.set_rhs(
        name,
        cas.DM(0)  # d_dot = 0 + process_noise (added by MHE)
    )
    # TODO: Allow alternative disturbance models

# Define measured variables and output expressions
for name in control_design["controlled_variables"]:
    i = system.output_names.index(name)
    model.set_meas(meas_name=name, expr=outputs[i])

model.setup()

print("\nModel setup complete.")
print(f"Total states: {model.n_x}")
print(f"  - Original system states: {system.n}")
print(f"  - Disturbance states: {len(control_design['unmeasured_disturbances'])}")
print(f"Manipulated inputs: {model.n_u}")
print(f"Controlled variables: {model.n_y}")

print("\nState variables:", list(model.x.keys()))
u_names = list(model.u.keys())
u_names.remove('default')
print("Mainpulated variables:", u_names)
y_names = list(model.y.keys())
y_names.remove('default')
print("Measured output variables:", y_names)

## Simulate MPC with Full State Measurement

In [10]:
# Re-build state vector after model.setup() called.
states = cas.vertcat(*[model.x[name] for name in system.state_names])

# Build input vector after model.setup() called.
inputs = []
for name in system.input_names:
    if name in control_design['manipulated_variables']:
        inputs.append(model.u[name])
    elif name in control_design.get('unmeasured_disturbances', []):
        inputs.append(model.x[name])
inputs = cas.vertcat(*inputs)

# Re-generate expressions from CasADi model functions
rhs = system.f(t, states, inputs)
outputs = system.h(t, states, inputs)

In [11]:
# ========================================
# 1. Setup MPC Controller
# ========================================
mpc = do_mpc.controller.MPC(model)

setup_mpc = {
    'n_horizon': 20,        # Prediction horizon (hours)
    't_step': 1.0,          # Time step (hours)
    'n_robust': 0,          # No robust horizon for now
    'store_full_solution': True,
}

mpc.set_param(**setup_mpc)

# ========================================
# 2. Define MPC Objective Function
# ========================================

# Setpoints for controlled variables
setpoints = {
    'tank_1_L': 1.5,         # Desired tank 1 level
    'tank_1_conc_out': 2.0,  # Desired tank 1 concentration
    'tank_2_L': 1.5,         # Desired tank 2 level
    'tank_2_conc_out': 2.0,  # Desired tank 2 concentration
    'mixer_conc_out': 2.0,   # Desired mixer concentration
}

# Tracking weights (higher = more important)
cv_weights = {
    'tank_1_L': 10.0,
    'tank_1_conc_out': 1.0,
    'tank_2_L': 10.0,
    'tank_2_conc_out': 1.0,
    'mixer_conc_out': 5.0,  # Most important
}

# Build objective: sum of squared tracking errors
mterm = 0  # Terminal cost
lterm = 0  # Stage cost

for cv_name in control_design['controlled_variables']:
    sp = setpoints[cv_name]
    weight = cv_weights[cv_name]
    cv_expr = outputs[system.output_names.index(cv_name)]
    error = cv_expr - sp
    mterm += weight * error ** 2
    lterm += weight * error ** 2

mpc.set_objective(mterm=cas.DM(0), lterm=lterm)

# Penalize control effort (optional)
mv_weights = {
    'tank_1_v_dot_in': 0.1,
    'tank_2_v_dot_in': 0.1,
    'mixer_r_1': 0.1,
}

rterm = 0
for mv_name in control_design['manipulated_variables']:
    weight = mv_weights[mv_name]
    rterm += weight * model.u[mv_name]**2

mpc.set_rterm(
    **{name: mv_weights[name] for name in control_design['manipulated_variables']}
)

In [12]:
# ========================================
# 3. Set MPC Constraints
# ========================================

# Input constraints
mpc.bounds['lower', '_u', 'tank_1_v_dot_in'] = 0.0
mpc.bounds['upper', '_u', 'tank_1_v_dot_in'] = 2.0

mpc.bounds['lower', '_u', 'tank_2_v_dot_in'] = 0.0
mpc.bounds['upper', '_u', 'tank_2_v_dot_in'] = 2.0

mpc.bounds['lower', '_u', 'mixer_r_1'] = 0.0
mpc.bounds['upper', '_u', 'mixer_r_1'] = 1.0

# State constraints (optional)
# Tank levels
mpc.bounds['lower', '_x', 'tank_1_L'] = 0.1
mpc.bounds['upper', '_x', 'tank_1_L'] = 3.0

mpc.bounds['lower', '_x', 'tank_2_L'] = 0.1
mpc.bounds['upper', '_x', 'tank_2_L'] = 3.0

# Disturbance bounds (reasonable ranges)
mpc.bounds['lower', '_x', 'tank_1_conc_in'] = 0.0
mpc.bounds['upper', '_x', 'tank_1_conc_in'] = 10.0

mpc.bounds['lower', '_x', 'tank_2_conc_in'] = 0.0
mpc.bounds['upper', '_x', 'tank_2_conc_in'] = 10.0

mpc.bounds['lower', '_x', 'mixer_v_dot_out'] = 0.0
mpc.bounds['upper', '_x', 'mixer_v_dot_out'] = 5.0

# Setup MPC
mpc.setup()

## Run Simulation to Test Control Performance

In [ ]:
# ========================================
# 4. Setup Simulator (Perfect Model)
# ========================================
simulator = do_mpc.simulator.Simulator(model)
simulator.set_param(t_step=1.0)  # Same as MPC
simulator.setup()

# ========================================
# 5. Setup Estimator (Perfect State Feedback for Testing)
# ========================================
# This only works when outputs are same as states
# estimator = do_mpc.estimator.StateFeedback(model)
# Note: no need for estimator.setup()

# ========================================
# 6. Set Initial Conditions
# ========================================

assert model.n_u == 3
assert model.n_x == 7
assert model.n_y == 5

# Initial state
x0_init = np.zeros((model.n_x, 1))

# System states
x0_init[0] = 1.0   # tank_1_L
x0_init[1] = 0.0  # tank_1_m
x0_init[2] = 1.0   # tank_2_L
x0_init[3] = 0.0  # tank_2_m

# Disturbance states (true values in simulation)
x0_init[4] = 1.5   # tank_1_conc_in (true value)
x0_init[5] = 2.5   # tank_2_conc_in (true value)
x0_init[6] = 0.8   # mixer_v_dot_out (true value)

# Set initial state for all components
mpc.x0 = x0_init
simulator.x0 = x0_init

# Calculate initial outputs
# Assumes no direct transmission from u(0) to y(0).
y0_init = simulator.model._meas_fun(
    simulator.x0,                           # _x: states
    np.zeros((simulator.model.n_u, 1)),     # _u: inputs
    np.zeros((simulator.model.n_z, 1)),     # _z: algebraic variables
    np.zeros((simulator.model.n_tvp, 1)),   # _tvp: time-varying parameters
    np.zeros((simulator.model.n_p, 1)),     # _p: parameters
    np.zeros((simulator.model.n_y, 1))      # _v: measurement noise
)

# Set initial guess for MPC
mpc.set_initial_guess()

# ========================================
# 7. Run Closed-Loop Simulation
# ========================================

n_steps = 100  # Simulate 100 hours

print("\nRunning closed-loop simulation...")
print(f"Simulation steps: {n_steps}")
print(f"Time step: {setup_mpc['t_step']} hours")

x0 = x0_init
for k in range(n_steps):
    # Get control action from MPC
    u0 = mpc.make_step(x0)

    # Simulate system
    y_next = simulator.make_step(u0)

    # For perfect controller testing, use true state from simulator
    x0 = simulator.x0

    if k % 10 == 0:
        print(f"Step {k}/{n_steps}")

print("Simulation complete!")

In [ ]:
import pandas as pd
sim_results = pd.concat({
    't': pd.DataFrame(simulator.data['_time']), 
    'U': pd.DataFrame(simulator.data['_u']),
    'X': pd.DataFrame(simulator.data['_x']),
    'Y': pd.DataFrame(simulator.data['_y'])
}, axis=1)
print(sim_results.head(5).round(2))

In [ ]:
# Correct way to inspect DO-MPC data
print("MPC data type:", type(mpc.data))
print("MPC data attributes:", [attr for attr in dir(mpc.data) if not attr.startswith('_')])

print("\n" + "="*60)
print("What's in the data:")
print("="*60)

# The data is accessed like a structured array
# Try accessing known variables
print("\nStates (_x):")
try:
    print("  Shape:", mpc.data['_x'].shape)
    print("  Variables:", [name for name in system.state_names])
except Exception as e:
    print(f"  Error: {e}")

print("\nInputs (_u):")
try:
    print("  Shape:", mpc.data['_u'].shape)
    print("  Variables:", control_design['manipulated_variables'])
except Exception as e:
    print(f"  Error: {e}")

print("\nTime:")
try:
    print("  Shape:", mpc.data['_time'].shape)
except Exception as e:
    print(f"  Error: {e}")

print("\n" + "="*60)
print("\nSimulator data:")
print("="*60)

print("\nStates (_x):")
try:
    print("  Shape:", simulator.data['_x'].shape)
except Exception as e:
    print(f"  Error: {e}")

print("\nAux (_aux):")
try:
    print("  Shape:", simulator.data['_aux'].shape)
    # Try to see what's in aux
    print("  First element:", simulator.data['_aux'][0])
except Exception as e:
    print(f"  Error: {e}")

print("\nTime:")
try:
    print("  Shape:", simulator.data['_time'].shape)
except Exception as e:
    print(f"  Error: {e}")

In [ ]:
# ========================================
# 8. Visualize Results
# ========================================

fig, ax = plt.subplots(3, 1, figsize=(12, 10))

# Get time vector
time = simulator.data['_time'].flatten()

# ========================================
# Plot 1: Controlled Variables (from simulator outputs)
# ========================================
ax[0].set_title('Controlled Variables (Outputs)')

# Get measurement data directly from simulator (shape is (100, n_y))
# The measurements are in the order defined by control_design['controlled_variables']
y_data = simulator.data['_y']

# Simulator.make_step stores y(k+1) in the '_y' field, not y(k).
# Therefore shift the values back forward one time step and insert
# the initial condition at t = 0.
y_data = np.roll(y_data, 1)
y_data[0, :] = np.array(y0_init).flatten()

# Map to CV names based on order in control_design
cv_names = control_design['controlled_variables']
cv_indices = {name: i for i, name in enumerate(cv_names)}

# Plot each CV
for cv_name in cv_names:
    idx = cv_indices[cv_name]
    ax[0].plot(time, y_data[:, idx], label=cv_name, marker='.')

# Add setpoints
for cv_name, sp in setpoints.items():
    ax[0].axhline(y=sp, color='gray', linestyle='--', alpha=0.5, linewidth=1)

ax[0].set_ylabel('CV Values')
ax[0].legend(loc='best')
ax[0].grid(True)

# ========================================
# Plot 2: Manipulated Variables
# ========================================
ax[1].set_title('Manipulated Variables (Inputs)')

# Get input data (shape is (100, 3))
u_data = mpc.data['_u']
mv_names = control_design['manipulated_variables']
mv_indices = {name: i for i, name in enumerate(mv_names)}

# Plot each MV
for mv_name in mv_names:
    idx = mv_indices[mv_name]
    ax[1].plot(time, u_data[:, idx], label=mv_name, marker='.')

ax[1].set_ylabel('MV Values')
ax[1].legend(loc='best')
ax[1].grid(True)

# ========================================
# Plot 3: Disturbances
# ========================================
ax[2].set_title('Disturbances (Unknown in Real Case)')

# Get disturbance states from state data
x_data = simulator.data['_x']
dist_names = control_design['unmeasured_disturbances']

# Disturbances are at the end of the state vector (after system states)
n_system_states = len(system.state_names)
dist_start_idx = n_system_states

# Plot each disturbance
for i, dist_name in enumerate(dist_names):
    dist_idx = dist_start_idx + i
    ax[2].plot(time, x_data[:, dist_idx], label=dist_name, marker='.')

ax[2].set_xlabel('Time (hours)')
ax[2].set_ylabel('Disturbance Values')
ax[2].legend(loc='best')
ax[2].grid(True)

plt.tight_layout()
plt.show()

# ========================================
# Performance Summary (using simulator outputs)
# ========================================
print("\n=== Performance Summary ===")

# Get final values from simulator measurements
y_final = y_data[-1, :]

for cv_name in cv_names:
    idx = cv_indices[cv_name]
    final_value = y_final[idx]
    sp = setpoints[cv_name]
    print(f"Final {cv_name}: {final_value:.3f} (target: {sp})")

In [ ]:
# ========================================
# 9. Print Performance Metrics
# ========================================

# Calculate CVs from state data
time = simulator.data['_time'].flatten()

tank_1_L = simulator.data['_x'][:, 0]
tank_1_m = simulator.data['_x'][:, 1]
tank_2_L = simulator.data['_x'][:, 2]
tank_2_m = simulator.data['_x'][:, 3]
mixer_v_dot_out = simulator.data['_x'][:, 6]
mixer_r_1 = mpc.data['_u'][:, 2]

# Calculate concentrations
tank_1_conc_out = tank_1_m / (19.635 * tank_1_L)
tank_2_conc_out = tank_2_m / (19.635 * tank_2_L)

# Calculate mixer concentration
mixer_v_dot_1 = mixer_r_1 * mixer_v_dot_out
mixer_v_dot_2 = (1 - mixer_r_1) * mixer_v_dot_out
mixer_conc_out = (mixer_v_dot_1 * tank_1_conc_out + mixer_v_dot_2 * tank_2_conc_out) / mixer_v_dot_out

# Store CVs in a dictionary
cv_data_dict = {
    'tank_1_L': tank_1_L,
    'tank_1_conc_out': tank_1_conc_out,
    'tank_2_L': tank_2_L,
    'tank_2_conc_out': tank_2_conc_out,
    'mixer_conc_out': mixer_conc_out
}

# Calculate tracking errors
tracking_errors = {}
for cv_name in control_design['controlled_variables']:
    cv_data = cv_data_dict[cv_name]
    sp = setpoints[cv_name]
    
    # RMS error
    rms_error = np.sqrt(np.mean((cv_data - sp)**2))
    tracking_errors[cv_name] = rms_error

print("\n=== Performance Metrics ===")
for cv_name, error in tracking_errors.items():
    print(f"{cv_name}: RMS error = {error:.4f}")

# Settling time (time to reach within 5% of setpoint)
print("\n=== Settling Times ===")
for cv_name in control_design['controlled_variables']:
    cv_data = cv_data_dict[cv_name]
    sp = setpoints[cv_name]
    tolerance = 0.05 * abs(sp)
    
    # Find first time within tolerance and stays within
    within_tolerance = np.abs(cv_data - sp) < tolerance
    if np.any(within_tolerance):
        settling_idx = np.argmax(within_tolerance)
        settling_time = settling_idx * setup_mpc['t_step']
        print(f"{cv_name}: {settling_time:.1f} hours")
    else:
        print(f"{cv_name}: Not settled within {tolerance:.3f}")

# Additional metrics
print("\n=== Final Values ===")
for cv_name in control_design['controlled_variables']:
    cv_data = cv_data_dict[cv_name]
    sp = setpoints[cv_name]
    final_value = cv_data[-1]
    final_error = final_value - sp
    print(f"{cv_name}: {final_value:.3f} (target: {sp:.1f}, error: {final_error:+.3f})")

# Control effort
print("\n=== Control Effort ===")
tank_1_v_dot_in = mpc.data['_u'][:, 0]
tank_2_v_dot_in = mpc.data['_u'][:, 1]
mixer_r_1_vals = mpc.data['_u'][:, 2]

print(f"tank_1_v_dot_in: mean={np.mean(tank_1_v_dot_in):.3f}, std={np.std(tank_1_v_dot_in):.3f}")
print(f"tank_2_v_dot_in: mean={np.mean(tank_2_v_dot_in):.3f}, std={np.std(tank_2_v_dot_in):.3f}")
print(f"mixer_r_1: mean={np.mean(mixer_r_1_vals):.3f}, std={np.std(mixer_r_1_vals):.3f}")

# Total variation (measure of control smoothness)
print("\n=== Control Smoothness (Total Variation) ===")
tv_1 = np.sum(np.abs(np.diff(tank_1_v_dot_in)))
tv_2 = np.sum(np.abs(np.diff(tank_2_v_dot_in)))
tv_r = np.sum(np.abs(np.diff(mixer_r_1_vals)))

print(f"tank_1_v_dot_in TV: {tv_1:.3f}")
print(f"tank_2_v_dot_in TV: {tv_2:.3f}")
print(f"mixer_r_1 TV: {tv_r:.3f}")

## Define State Estimator - Extended Kalman Filter (EKF)

In [ ]:
# Setup Estimator
estimator_design = {
    'meas_noise_std': {
        'tank_1_L': 0.1, 
        'tank_1_conc_out': 0.5, 
        'tank_2_L': 0.1, 
        'tank_2_conc_out': 0.5, 
        'mixer_conc_out': 0.5
    },
    'process_noise_std': {
        'tank_1_L': 0.01,
        'tank_1_m': 0.01,
        'tank_2_L': 0.01, 
        'tank_2_m': 0.01,
        'tank_1_conc_in': 1.0, 
        'tank_2_conc_in': 1.0, 
        'mixer_v_dot_out': 1.0
    },
    'initial_state_covariance': {
        'tank_1_L': 1000.0,
        'tank_1_m': 1000.0,
        'tank_2_L': 1000.0, 
        'tank_2_m': 1000.0,
        'tank_1_conc_in': 1000.0,
        'tank_2_conc_in': 1000.0,
        'mixer_v_dot_out': 1000.0,
    },
    'kwargs': {
        't_step': 1.0,             # Time step (hours)
    }
}

# ========================================
# Create EKF estimator
# ========================================
estimator = do_mpc.estimator.EKF(model)

# ========================================
# Build measurement noise covariance matrix (R)
# ========================================
# R is a diagonal matrix with measurement noise variances
R = np.diag(
    [
        estimator_design['meas_noise_std'][name] ** 2
        for name in control_design['controlled_variables']
    ]
)
print(f"Measurement noise covariance (R) shape: {R.shape}")

# ========================================
# Build process noise covariance matrix (Q)
# ========================================
# Q is a diagonal matrix with process noise variances
n_x = model.n_x
Q = np.diag(
    [
        estimator_design['process_noise_std'][name] ** 2
        for name in model.x.keys()
    ]
)
print(f"Process noise covariance (Q) shape: {Q.shape}")

# ========================================
# Build initial state covariance matrix (P0)
# ========================================
# P0 represents uncertainty in initial state estimate
P0 = np.diag(
    [
        estimator_design['initial_state_covariance'][name]
        for name in model.x.keys()
    ]
)
print(f"Initial covariance (P0) shape: {P0.shape}")

# ========================================
# Setup the EKF with covariances
# ========================================
estimator.settings.t_step = estimator_design['kwargs']['t_step']
estimator.settings.P_x = P0   # Initial state covariance
estimator.settings.P_v = Q    # Process noise covariance  
estimator.settings.P_w = R    # Measurement noise covariance

# Setup the estimator
estimator.setup()

# Set initial state estimate
x0 = np.zeros((model.n_x, 1))

estimator.x0 = x0
estimator.reset_history()

print("\nEKF setup complete.")
print(f"Time step: {estimator_design['kwargs']['t_step']} hours")
print(f"Number of states: {estimator.model.n_x}")
print(f"Number of known inputs: {estimator.model.n_u}")
print(f"Number of measurements: {estimator.model.n_y}")


## Generate Simulated Input-Output Data to Test Estimator

In [ ]:
dt = 1.0
system_dt = StateSpaceModelDTFromCT(system, dt=dt)
system_dt

nT = 100
simulate = make_n_step_simulation_function_from_model(system_dt, nT)
simulate

In [ ]:
# Example: Run estimator in a loop
for k in range(100):
    # Get measurements (from real system or simulator)
    y_measured = np.random.randn(n_y, 1) * 0.1 + 1.0  # Example
    
    # Get control inputs
    u_applied = np.array([[0.5], [0.5], [0.6]])  # Example
    
    # Run EKF to estimate states (including disturbances)
    x_estimated = estimator.make_step(y_measured)
    
    # Extract disturbance estimates
    dist_estimates = {}
    for i, dist_name in enumerate(d.keys()):
        state_idx = len(x) + i  # Disturbances come after system states
        dist_estimates[dist_name] = float(x_estimated[state_idx])
    
    print(f"Step {k}: Disturbances = {dist_estimates}")